# Assignment: Orchestrate Conversation with Dynamic Prompt Injection


---

### Objective:
This assignment focuses on a crucial advanced technique in LLM applications: **dynamic prompt injection**. You'll build a conversational system that adapts its prompts based on user input, retrieved context, or internal logic, demonstrating how to maintain coherence and provide relevant responses by injecting information into the LLM's working memory or instructions *mid-conversation*.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API. **OpenAI's models (e.g., GPT-4o, GPT-4, GPT-3.5-turbo)** are recommended for their strong performance. Configure your API key securely (e.g., via environment variables).
2.  **Environment Setup**: Install the necessary Python libraries: `pip install openai python-dotenv`.
3.  **Scenario**: You will build a simple **"Product Inquiry Assistant"**. This assistant will:
    * Receive user questions about products.
    * **Dynamically inject** product details (from a simulated database/dictionary) into the prompt before querying the LLM.
    * Handle follow-up questions, always considering the current product context.
    * Allow the user to switch products, triggering a new dynamic injection.
4.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
5.  **Analysis**: Explain how dynamic prompt injection works, its benefits, and challenges.

---

## Part 1: Setup and LLM Configuration
Begin by setting up your environment and configuring your LLM.

### Task 1.1: Install Libraries and Configure LLM
Install `openai` and `python-dotenv`. Set up your OpenAI API key from environment variables.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install openai python-dotenv --quiet

import openai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- IMPORTANT: Create a .env file in the same directory as this notebook with the line: ---
# OPENAI_API_KEY="YOUR_OPENAI_API_KEY_HERE"

# Configure OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    print("WARNING: OPENAI_API_KEY not found in environment variables. Please set it in .env file.")
else:
    print("OpenAI API key loaded!")

# Define the LLM model to use
LLM_MODEL = "gpt-3.5-turbo" # You can use "gpt-4o", "gpt-4", etc. if you have access

print(f"Using LLM Model: {LLM_MODEL}")

### Task 1.2: Simulate a Product Database
Create a Python dictionary to represent a simple product database. This will be the source of information for dynamic injection.

In [ ]:
product_database = {
    "laptop": {
        "name": "ProBook X1",
        "specs": "16-inch Retina display, M3 Pro chip, 16GB RAM, 512GB SSD, 12-hour battery life.",
        "features": "Lightweight aluminum unibody, advanced cooling system, backlit keyboard, Thunderbolt 4 ports.",
        "price": "$1999",
        "warranty": "1-year limited warranty."
    },
    "smartphone": {
        "name": "NexPhone Ultra",
        "specs": "6.7-inch OLED display, A10 Bionic chip, 8GB RAM, 256GB storage, triple-lens camera system.",
        "features": "Water and dust resistant, face unlock, fast charging, 5G connectivity.",
        "price": "$999",
        "warranty": "1-year manufacturer warranty."
    },
    "headphones": {
        "name": "SoundWave Pro",
        "specs": "Over-ear design, Active Noise Cancellation (ANC), Bluetooth 5.2, 30-hour battery life.",
        "features": "Comfortable earcups, foldable design, ambient sound mode, custom EQ settings via app.",
        "price": "$299",
        "warranty": "6-month limited warranty."
    }
}

print("Product database simulated with:", list(product_database.keys()))

---

## Part 2: Build the Conversational Agent with Dynamic Prompt Injection
You'll create a function that manages the conversation, identifies the current product context, and dynamically injects relevant details into the LLM's prompt.

### Task 2.1: Implement `get_product_context` Function
This function will identify if a product mention exists in the user's query and retrieve its details from the `product_database`.

In [ ]:
def get_product_context(query: str, current_product: str = None) -> tuple:
    """
    Identifies a product in the query and returns its details.
    Prioritizes explicit mentions, otherwise uses the current product context.
    Returns (product_name, product_details_string).
    """
    detected_product = None
    for product_key in product_database.keys():
        if product_key in query.lower():
            detected_product = product_key
            break

    if detected_product:
        product_name = product_database[detected_product]['name']
        details = product_database[detected_product]
        product_details_string = f"Product Name: {details['name']}\nSpecs: {details['specs']}\nFeatures: {details['features']}\nPrice: {details['price']}\nWarranty: {details['warranty']}"
        return detected_product, product_details_string
    elif current_product and current_product in product_database:
        # If no new product detected, use the current conversation product
        details = product_database[current_product]
        product_details_string = f"Product Name: {details['name']}\nSpecs: {details['specs']}\nFeatures: {details['features']}\nPrice: {details['price']}\nWarranty: {details['warranty']}"
        return current_product, product_details_string
    else:
        return None, None

print("get_product_context function defined!")

### Task 2.2: Implement `get_llm_response` Function
This function will interact with the OpenAI API. It will dynamically build the prompt based on the user's query and the injected product context.

In [ ]:
async def get_llm_response(conversation_history: list, product_context_string: str = None) -> str:
    """
    Generates a response from the LLM, dynamically injecting product context.
    """
    system_message = {
        "role": "system",
        "content": "You are a helpful product inquiry assistant. Your goal is to answer questions about products based on the provided product details. If a question is outside the scope of the provided product details or unrelated to products, kindly state that you can only assist with product inquiries. Always be polite and concise."
    }

    # Dynamically inject product context at the beginning of the conversation history
    # This acts like a 'system' message but can be updated dynamically.
    dynamic_context_message = {
        "role": "system",
        "content": f"CURRENT PRODUCT DETAILS:\n{product_context_string}\n\nAnswer user questions strictly based on these details. If a question is about a different product, ask the user to specify which product they are interested in."
        if product_context_string else
        "No specific product context provided. Please ask the user to specify a product (e.g., laptop, smartphone, headphones) before answering product-specific questions."
    }

    # Combine dynamic context with the rest of the conversation history
    messages = [system_message, dynamic_context_message] + conversation_history

    try:
        response = await openai.chat.completions.create(
            model=LLM_MODEL,
            messages=messages,
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content
    except openai.APIError as e:
        print(f"OpenAI API error: {e}")
        return "I'm sorry, I'm having trouble connecting to the AI right now. Please try again later."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "An unexpected error occurred. Please try again."

print("get_llm_response function defined!")

### Task 2.3: Implement the Main Conversational Loop
Create an asynchronous function that handles the entire conversational flow, managing the `current_product` state and calling the other functions.

In [ ]:
async def run_product_assistant():
    conversation_history = []
    current_product_key = None
    product_details_string = None

    print("Welcome to the Product Inquiry Assistant! You can ask about: laptop, smartphone, headphones.")
    print("Type 'exit' to end the conversation.")
    print("--------------------------------------------------------------------------------------")

    while True:
        user_query = input("\nYour question: ")
        if user_query.lower() == 'exit':
            print("Thank you for using the Product Inquiry Assistant. Goodbye!")
            break

        # 1. Detect if a new product is mentioned or use existing context
        detected_product, new_product_details_string = get_product_context(user_query, current_product_key)

        if detected_product and detected_product != current_product_key:
            print(f"Assistant: Switching context to {product_database[detected_product]['name']}.")
            current_product_key = detected_product
            product_details_string = new_product_details_string
            # Reset conversation history when product context changes significantly
            conversation_history = []
        elif detected_product is None and current_product_key is None:
            print("Assistant: Please specify which product you are interested in (e.g., laptop, smartphone, headphones).")
            continue # Skip LLM call if no product is specified initially

        # Add user query to conversation history
        conversation_history.append({"role": "user", "content": user_query})

        # 2. Get LLM response with dynamically injected product context
        llm_response = await get_llm_response(conversation_history, product_details_string)

        # Add LLM response to conversation history
        conversation_history.append({"role": "assistant", "content": llm_response})

        print(f"Assistant: {llm_response}")

print("run_product_assistant function defined!")

---

## Part 3: Test the Conversational Agent
Run your `run_product_assistant` function and interact with it. Observe how the context shifts and how the LLM responds based on the injected data.

In [ ]:
import asyncio

# Run the assistant
await run_product_assistant()

---

## Part 4: Analysis and Reflection
Based on your interaction with the bot, answer the following questions.

### Task 4.1: Dynamic Prompt Injection Mechanism
* **How it Works**: Describe in detail how dynamic prompt injection is implemented in your `get_llm_response` function. How does the `product_context_string` become part of the LLM's understanding?
* **Location of Injection**: Why was the `dynamic_context_message` placed at the beginning of the `messages` list, right after the initial `system_message`? What would happen if it was placed at the end, or if the `conversation_history` was not reset when switching products?
* **Product Switching**: Explain how your system detects a change in the user's desired product and how it adapts the context accordingly.

### Task 4.2: Benefits and Use Cases
* **Advantages**: What are the key benefits of using dynamic prompt injection in conversational AI, as demonstrated by this assignment? How does it improve the LLM's relevance and accuracy?
* **Alternative Methods**: Briefly discuss an alternative to dynamic prompt injection for managing context (e.g., fine-tuning, retrieval-augmented generation (RAG)). When might dynamic prompt injection be preferred over these alternatives?
* **Real-world Applications**: Beyond product inquiries, what are other real-world scenarios where dynamic prompt injection would be highly beneficial (e.g., personalized tutors, technical support, content generation based on user preferences)?

### Task 4.3: Challenges and Limitations
* **Prompt Size**: What potential issues could arise if the injected dynamic context becomes very large (e.g., hundreds or thousands of tokens)? How might this impact performance or cost?
* **Contextual Overlap/Conflict**: What challenges might arise if the dynamic context contradicts information already present in the conversation history or if the LLM struggles to prioritize the injected context?
* **Scalability**: How would you scale this approach if you had millions of products or highly complex product details? What mechanisms would you need beyond a simple dictionary?


---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Make sure your `.env` file (or equivalent API key setup) is mentioned but **NOT** included in the submitted notebook for security reasons.
* Save your Jupyter Notebook as `[YourName]_Dynamic_Prompt_Injection_Assignment.ipynb`.